In [ ]:
## script to generate Scispot registrations for ChESS 96-well plate
## lkp 2022/11/15

import pandas as pd
import requests
import json
import sys
import datetime

sys.stdout.write("Imported required packages")

api_key_file = 'G:/My Drive/Lindsay Pino/proj/2023_scispot_utils/data/scispot_api_key.txt'
API_KEY = open(api_key_file, 'r').readlines()[0]
#print(API_KEY)

# Jupyter notebook in Scispot

- Scispot --> LabSheets --> Jupyter --> +New --> write script/notebook
  - ~~need jupyterhub server credentials?~~

- ? can I look up a UUID by sample name?
  - update-by-id to fetch UUID from Sample ID
- ? can I look up UUIDs by manifest name?
  - not yet! 
- ? can I specify other "variable" inputs/params? or can scientist edit the ipynb with those values?
  - A: yes, can code input parameters 
  - ? can I read in other sheets into pandas?
    - ~~nm don't worry about this for now...~~ done, see below


### tips
- update-by-id to fetch UUID from Sample ID 
- https://www.loom.com/share/32c01c2cb1294720b15596b0a43af5e9

In [2]:
# lookup an entry by UUID
def find_entry_from_uuid(manager, uuid_list):
    session = requests.Session()
    url = "https://api.scispot.io/tryingtofixcors/labsheets/find-row"
    payload = {
        "apiKey": API_KEY,
        "manager": manager,
        "uuid": uuid_list
    }
    ret = session.post(url, json=payload)
    return json.loads(ret.text)

test_uuid = "0344a99d-888a-443e-9a23-13eceb279ae1"
find_entry_from_uuid('Sample Manager', test_uuid)

{'headers': ['UUID',
  'Sample ID',
  'Working compound',
  'Working cell line',
  'Pert Dose',
  'Pert Time',
  'Pert Protocol',
  'Tissue ID',
  'Tissue Passage',
  'Type',
  'Parent',
  'Children',
  'Location',
  'Storage Form',
  'Storage Amount',
  'Pass',
  'Quantity Number of Cells (x1000)',
  'Created By'],
 'row': ['0344a99d-888a-443e-9a23-13eceb279ae1',
  'THP1_dbgen1_VTP50469_3',
  'VTP50469_1',
  'THP1_dbgen1',
  '',
  '',
  '',
  '',
  '',
  'Frozen Nuclei',
  '90b19820-726d-4086-b74a-199d37899ea5',
  '',
  'ChESS_dbgen1_202212_plate1rep3',
  '',
  '',
  '',
  '',
  ''],
 'success': 'true'}

In [3]:
# turn json into pandas df
def json_to_pd_df(json_in):
    df_out = pd.DataFrame(data=json_in['row']).T
    df_out.columns = list(json_in['headers'])
    return df_out

temp = find_entry_from_uuid('Sample Manager', test_uuid)
json_to_pd_df(temp)

,UUID,Sample ID,Working compound,Working cell line,Pert Dose,Pert Time,Pert Protocol,Tissue ID,Tissue Passage,Type,Parent,Children,Location,Storage Form,Storage Amount,Pass,Quantity Number of Cells (x1000),Created By
0,0344a99d-888a-443e-9a23-13eceb279ae1,THP1_dbgen1_VTP50469_3,VTP50469_1,THP1_dbgen1,,,,,,Frozen Nuclei,90b19820-726d-4086-b74a-199d37899ea5,,ChESS_dbgen1_202212_plate1rep3,,,,,


In [4]:
# fetch any row from the Protein Manager to get the columns
def fetch_random_entry():
    session = requests.Session()
    url = "https://api.scispot.io/tryingtofixcors/labsheets/list-rows"
    payload = {
        "apiKey": API_KEY,
        "manager": "Sample Manager",
        "pageSize": "1",
        "page": "1"
    }
    ret = session.post(url, json=payload)
    return json.loads(ret.text)

# initialize an empty Protein Manager
prot_df = pd.DataFrame(data=fetch_random_entry()['rows'], columns=fetch_random_entry()['headers'])
prot_df= prot_df.drop(prot_df.index[0])
prot_df

,UUID,Sample ID,Working compound,Working cell line,Pert Dose,Pert Time,Pert Protocol,Tissue ID,Tissue Passage,Type,Parent,Children,Location,Storage Form,Storage Amount,Pass,Quantity Number of Cells (x1000),Created By


In [5]:
def fetch_entry_from_id(manager, sample_id):
    session = requests.Session()
    url = "https://api.scispot.io/tryingtofixcors/labsheets/find-row-by-id"
    payload = {
        "apiKey": API_KEY,
        "labsheet": manager,
        "id": sample_id
    }
    ret = session.post(url, json=payload)
    return json.loads(ret.text)

test_sampleid = "THP1_dbgen1_VTP50469_3"
fetch_entry_from_id("Sample Manager",test_sampleid)

{'rows': [['0344a99d-888a-443e-9a23-13eceb279ae1',
   'THP1_dbgen1_VTP50469_3',
   'VTP50469_1',
   'THP1_dbgen1',
   '',
   '',
   '',
   '',
   '',
   'Frozen Nuclei',
   '90b19820-726d-4086-b74a-199d37899ea5',
   '',
   'ChESS_dbgen1_202212_plate1rep3',
   '',
   '',
   '',
   '',
   ''],
  ['83d5ede7-4fb0-4659-8b74-a0ed01e63175',
   'THP1_dbgen1_VTP50469_3',
   'VTP50469_1',
   'THP1_dbgen1',
   '',
   '',
   '',
   '',
   '',
   '',
   '90b19820-726d-4086-b74a-199d37899ea5',
   '',
   'ChESS_dbgen1_202212_plate1rep3',
   '',
   '',
   '',
   '0.000',
   '']],
 'headers': ['UUID',
  'Sample ID',
  'Working compound',
  'Working cell line',
  'Pert Dose',
  'Pert Time',
  'Pert Protocol',
  'Tissue ID',
  'Tissue Passage',
  'Type',
  'Parent',
  'Children',
  'Location',
  'Storage Form',
  'Storage Amount',
  'Pass',
  'Quantity Number of Cells (x1000)',
  'Created By'],
 'success': True}

In [6]:
# add a new row to the Protein Manager
def add_protein_manager_entry(new_row):
    url = "https://api.scispot.io/tryingtofixcors/labsheets/add-rows"
    payload = {
        "apiKey": API_KEY,
        "manager": "Protein Manager",
        "rows": [new_row]
    }
    headers = {"Content-Type": "application/json"}
    print(new_row)
    response = requests.request("POST", url, json=payload, headers=headers)
    print(response.text)
    
# to add rows to Sample Manager, some columns required:
test_new_row_this_works = [
      "proteinid_here3",# Protein ID
      "0344a99d-888a-443e-9a23-13eceb279ae1", # Sample ID required, must be UUID
      "e45d66e9-abed-4d24-9dd8-8f074233c2d8", # Protocol ID required, must be UUID
      "01/01/2023", # "Extraction Date" column required, must be MM/DD/YYYY
      "Lindsay", # "Extraction Scientist" column required, must be name
      "extractionfrx_here", # Extraction fraction
      "extractionnote_here", # Extraction Note
      "0", # Protein Concentration (ug/mL)
      "bca_here", # BCA Results
      "", # Location
      "storageform_here", # Storage Form
      "storageamt_here", # Storage Amount (uL)
      "False", # "Pass" column required
      "", # Parent UUID
      "", # Children UUID
      "", # Created By
     "" # Creation Date
]

#add_entry(test_new_row_this_works)
test_new_row_this_works

['proteinid_here3',
 '0344a99d-888a-443e-9a23-13eceb279ae1',
 'e45d66e9-abed-4d24-9dd8-8f074233c2d8',
 '01/01/2023',
 'Lindsay',
 'extractionfrx_here',
 'extractionnote_here',
 '0',
 'bca_here',
 '',
 'storageform_here',
 'storageamt_here',
 'False',
 '',
 '',
 '',
 '']

In [7]:
## TODO check if row added? THP1_dbgen1_VTP50469_3
# uuids that geenrated from add_entry to protein manager:
# 0344a99d-888a-443e-9a23-13eceb279ae1 - Sample Manager and Protein Manager and Peptide Manager and ...??
# 90b19820-726d-4086-b74a-199d37899ea5 - Sample Manager and Protein Manager and Peptide Manager and ...??
# 98f8e123-84ab-4171-b26e-6c3c0ad6ecf3 - Sample Manager and Protein Manager and Peptide Manager and ...??
# f83d6043-0901-4362-bef9-324987e06451 - doesn't exist
# 485546de-f451-46cd-afbb-7bad60e998d5
# e4695003-2074-458c-8080-9a659bc208c5

find_entry_from_uuid("Protein Manager", "e4695003-2074-458c-8080-9a659bc208c5")

{'headers': ['UUID',
  'Protein ID',
  'Sample ID',
  'Protocol ID',
  'Extraction Date',
  'Extraction Scientist',
  'Extraction Fraction',
  'Extraction Notes',
  'Protein Concentration (ug/mL)',
  'BCA Results',
  'Location',
  'Storage Form',
  'Storage Amount (uL)',
  'Pass',
  'Parent',
  'Children',
  'Created By',
  'Creation Date'],
 'row': ['e4695003-2074-458c-8080-9a659bc208c5',
  'proteinid_here3',
  'THP1_dbgen1_VTP50469_3',
  'dummy_protocol',
  '01/01/2023',
  'Lindsay',
  'extractionfrx_here',
  'extractionnote_here',
  '0',
  'bca_here',
  '',
  'storageform_here',
  'storageamt_here',
  'False',
  '',
  '',
  '',
  ''],
 'success': 'true'}

In [ ]:
# someday I should actually make objects for scispot stuff...
# TODO: read in headers from API? so that if they change, this isn't all hardcoded...
class scispot_managers:
    def __init__( self, data, ):
        self.sample = pd.DataFrame()
        self.protein = pd.DataFrame()
        self.peptide = pd.DataFrame()
        
    def sample_to_protein( self, input ):
        self.peptide[ 'new_column' ] = input

def new_protein_row(sample_uuid, fraction):
    
    #print(sample_uuid)
    
    # generate protein ID from sample ID
    sample_id = json_to_pd_df(find_entry_from_uuid("Sample Manager", sample_uuid))['Sample ID'][0]
    #print(sample_id)
    protein_id = sample_id + "_" + fraction
    
    #for fx in extraction_fx:
          
    new_protein_row = [
        protein_id, # Protein ID
        sample_uuid, # Sample ID required, must be UUID
        "e45d66e9-abed-4d24-9dd8-8f074233c2d8", # Protocol ID required, must be UUID
        "01/01/2023", # "Extraction Date" column required, must be MM/DD/YYYY
        "Lindsay", # "Extraction Scientist" column required, must be name
        fraction, # Extraction fraction
        "extractionnote_here", # Extraction Note
        "0", # Protein Concentration (ug/mL)
        "bca_here", # BCA Results
        "", # Location
        "storageform_here", # Storage Form
        "storageamt_here", # Storage Amount (uL)
        "True", # "Pass" column required
        sample_uuid, # Parent UUID
        "", # Children UUID
        "", # Created By
        "" # Creation Date
    ]

    return(new_protein_row)

# read in a Scispot plate TSV export of Sample Manager
## TODO possible to pull a plate of samples with some manifest name/location from API?

# ChESS_dbgen1_plate1 was done manually
#sample_fi = "G:/My Drive/Lindsay Pino/proj/2023_scispot_utils/data/Sample-2023-04-10T19_24_09.tsv" # ChESS_dbgen1_202212_plate1rep2 
sample_fi = "G:/My Drive/Lindsay Pino/proj/2023_scispot_utils/data/Sample-2023-04-10T19_24_09.tsv" # ChESS_dbgen1_202212_plate1rep3
input_df = pd.read_table(sample_fi, sep='\t', engine="python")
input_df.tail()

## GENERATE PROTEIN SAMPLES
# generate the new protein samples
# note: uuid header string parses weird > "ï»¿uuid"
for uuid in input_df["ï»¿uuid"]:

    # define Protein fractions to be generated for each Sample
    extraction_frx = ['cytosol',
                     'chromatin',
                     'insoluble']
    
    # generate the Protein Manager row for each fraction
    for frx in extraction_frx:
        new_row = new_protein_row(uuid, frx)
    
        # add the row to the Protein Manager
        #add_protein_manager_entry(new_row)


cytosol
['THP1_dbgen1_VE821_2_cytosol', '7224b0bb-59d5-45b9-8a26-1a5acfe61d5e', 'e45d66e9-abed-4d24-9dd8-8f074233c2d8', '01/01/2023', 'Lindsay', 'cytosol', 'extractionnote_here', '0', 'bca_here', '', 'storageform_here', 'storageamt_here', 'True', '7224b0bb-59d5-45b9-8a26-1a5acfe61d5e', '', '', '']
chromatin
['THP1_dbgen1_VE821_2_chromatin', '7224b0bb-59d5-45b9-8a26-1a5acfe61d5e', 'e45d66e9-abed-4d24-9dd8-8f074233c2d8', '01/01/2023', 'Lindsay', 'chromatin', 'extractionnote_here', '0', 'bca_here', '', 'storageform_here', 'storageamt_here', 'True', '7224b0bb-59d5-45b9-8a26-1a5acfe61d5e', '', '', '']
insoluble
['THP1_dbgen1_VE821_2_insoluble', '7224b0bb-59d5-45b9-8a26-1a5acfe61d5e', 'e45d66e9-abed-4d24-9dd8-8f074233c2d8', '01/01/2023', 'Lindsay', 'insoluble', 'extractionnote_here', '0', 'bca_here', '', 'storageform_here', 'storageamt_here', 'True', '7224b0bb-59d5-45b9-8a26-1a5acfe61d5e', '', '', '']
cytosol
['THP1_dbgen1_Olaparib_2_cytosol', '6afb4d00-17d3-4a72-815a-6f48853288fe', 'e45d66e

['OVCAR3_dbgen1_K7844653_2_cytosol', 'd97ef0e6-b74b-4cfb-8e25-1f51f09a25f2', 'e45d66e9-abed-4d24-9dd8-8f074233c2d8', '01/01/2023', 'Lindsay', 'cytosol', 'extractionnote_here', '0', 'bca_here', '', 'storageform_here', 'storageamt_here', 'True', 'd97ef0e6-b74b-4cfb-8e25-1f51f09a25f2', '', '', '']
chromatin
['OVCAR3_dbgen1_K7844653_2_chromatin', 'd97ef0e6-b74b-4cfb-8e25-1f51f09a25f2', 'e45d66e9-abed-4d24-9dd8-8f074233c2d8', '01/01/2023', 'Lindsay', 'chromatin', 'extractionnote_here', '0', 'bca_here', '', 'storageform_here', 'storageamt_here', 'True', 'd97ef0e6-b74b-4cfb-8e25-1f51f09a25f2', '', '', '']
insoluble
['OVCAR3_dbgen1_K7844653_2_insoluble', 'd97ef0e6-b74b-4cfb-8e25-1f51f09a25f2', 'e45d66e9-abed-4d24-9dd8-8f074233c2d8', '01/01/2023', 'Lindsay', 'insoluble', 'extractionnote_here', '0', 'bca_here', '', 'storageform_here', 'storageamt_here', 'True', 'd97ef0e6-b74b-4cfb-8e25-1f51f09a25f2', '', '', '']
cytosol
['OVCAR3_dbgen1_51211288_2_cytosol', '2e8e4680-aff2-40d6-ae6f-a79e70b08054',

['CAOV3_dbgen1_K7845943_2_cytosol', '30a2fa5a-aae8-49e7-9a2c-a3eb7af80041', 'e45d66e9-abed-4d24-9dd8-8f074233c2d8', '01/01/2023', 'Lindsay', 'cytosol', 'extractionnote_here', '0', 'bca_here', '', 'storageform_here', 'storageamt_here', 'True', '30a2fa5a-aae8-49e7-9a2c-a3eb7af80041', '', '', '']
chromatin
['CAOV3_dbgen1_K7845943_2_chromatin', '30a2fa5a-aae8-49e7-9a2c-a3eb7af80041', 'e45d66e9-abed-4d24-9dd8-8f074233c2d8', '01/01/2023', 'Lindsay', 'chromatin', 'extractionnote_here', '0', 'bca_here', '', 'storageform_here', 'storageamt_here', 'True', '30a2fa5a-aae8-49e7-9a2c-a3eb7af80041', '', '', '']
insoluble
['CAOV3_dbgen1_K7845943_2_insoluble', '30a2fa5a-aae8-49e7-9a2c-a3eb7af80041', 'e45d66e9-abed-4d24-9dd8-8f074233c2d8', '01/01/2023', 'Lindsay', 'insoluble', 'extractionnote_here', '0', 'bca_here', '', 'storageform_here', 'storageamt_here', 'True', '30a2fa5a-aae8-49e7-9a2c-a3eb7af80041', '', '', '']
cytosol
['CAOV3_dbgen1_K7844653_2_cytosol', '4b68c9e0-ab50-4ee6-a9b4-e753e520ebed', 'e4

['SKNAS_dbgen1_T0011879_2_cytosol', '44a534a5-970f-4e7f-b8a9-602d38c940ff', 'e45d66e9-abed-4d24-9dd8-8f074233c2d8', '01/01/2023', 'Lindsay', 'cytosol', 'extractionnote_here', '0', 'bca_here', '', 'storageform_here', 'storageamt_here', 'True', '44a534a5-970f-4e7f-b8a9-602d38c940ff', '', '', '']
chromatin
['SKNAS_dbgen1_T0011879_2_chromatin', '44a534a5-970f-4e7f-b8a9-602d38c940ff', 'e45d66e9-abed-4d24-9dd8-8f074233c2d8', '01/01/2023', 'Lindsay', 'chromatin', 'extractionnote_here', '0', 'bca_here', '', 'storageform_here', 'storageamt_here', 'True', '44a534a5-970f-4e7f-b8a9-602d38c940ff', '', '', '']
insoluble
['SKNAS_dbgen1_T0011879_2_insoluble', '44a534a5-970f-4e7f-b8a9-602d38c940ff', 'e45d66e9-abed-4d24-9dd8-8f074233c2d8', '01/01/2023', 'Lindsay', 'insoluble', 'extractionnote_here', '0', 'bca_here', '', 'storageform_here', 'storageamt_here', 'True', '44a534a5-970f-4e7f-b8a9-602d38c940ff', '', '', '']
cytosol
['SKNAS_dbgen1_T0011026_2_cytosol', '473e0b2a-1fb1-41ed-a6e0-0ecac900b273', 'e4

['RH4_dbgen1_NMIAA_2_insoluble', '9c837265-0166-4b33-86f7-096ec7a7edb4', 'e45d66e9-abed-4d24-9dd8-8f074233c2d8', '01/01/2023', 'Lindsay', 'insoluble', 'extractionnote_here', '0', 'bca_here', '', 'storageform_here', 'storageamt_here', 'True', '9c837265-0166-4b33-86f7-096ec7a7edb4', '', '', '']
cytosol
['RH4_dbgen1_K7843670_2_cytosol', '8a2787fd-9aff-4224-9d6a-de489e9e5bd5', 'e45d66e9-abed-4d24-9dd8-8f074233c2d8', '01/01/2023', 'Lindsay', 'cytosol', 'extractionnote_here', '0', 'bca_here', '', 'storageform_here', 'storageamt_here', 'True', '8a2787fd-9aff-4224-9d6a-de489e9e5bd5', '', '', '']
chromatin
['RH4_dbgen1_K7843670_2_chromatin', '8a2787fd-9aff-4224-9d6a-de489e9e5bd5', 'e45d66e9-abed-4d24-9dd8-8f074233c2d8', '01/01/2023', 'Lindsay', 'chromatin', 'extractionnote_here', '0', 'bca_here', '', 'storageform_here', 'storageamt_here', 'True', '8a2787fd-9aff-4224-9d6a-de489e9e5bd5', '', '', '']
insoluble
['RH4_dbgen1_K7843670_2_insoluble', '8a2787fd-9aff-4224-9d6a-de489e9e5bd5', 'e45d66e9-ab

['RD_dbgen1_K7843631_2_cytosol', 'baaaa299-7b00-4868-981a-e14b4a6cf8ae', 'e45d66e9-abed-4d24-9dd8-8f074233c2d8', '01/01/2023', 'Lindsay', 'cytosol', 'extractionnote_here', '0', 'bca_here', '', 'storageform_here', 'storageamt_here', 'True', 'baaaa299-7b00-4868-981a-e14b4a6cf8ae', '', '', '']
chromatin
['RD_dbgen1_K7843631_2_chromatin', 'baaaa299-7b00-4868-981a-e14b4a6cf8ae', 'e45d66e9-abed-4d24-9dd8-8f074233c2d8', '01/01/2023', 'Lindsay', 'chromatin', 'extractionnote_here', '0', 'bca_here', '', 'storageform_here', 'storageamt_here', 'True', 'baaaa299-7b00-4868-981a-e14b4a6cf8ae', '', '', '']
insoluble
['RD_dbgen1_K7843631_2_insoluble', 'baaaa299-7b00-4868-981a-e14b4a6cf8ae', 'e45d66e9-abed-4d24-9dd8-8f074233c2d8', '01/01/2023', 'Lindsay', 'insoluble', 'extractionnote_here', '0', 'bca_here', '', 'storageform_here', 'storageamt_here', 'True', 'baaaa299-7b00-4868-981a-e14b4a6cf8ae', '', '', '']
cytosol
['RD_dbgen1_K7843670_2_cytosol', '2140be27-a1ca-4985-be26-28e6cbe51016', 'e45d66e9-abed-

['OVCAR3_dbgen1_K7845698_2_chromatin', 'b3f3d1d3-eb51-4615-a6e8-156e1a351db0', 'e45d66e9-abed-4d24-9dd8-8f074233c2d8', '01/01/2023', 'Lindsay', 'chromatin', 'extractionnote_here', '0', 'bca_here', '', 'storageform_here', 'storageamt_here', 'True', 'b3f3d1d3-eb51-4615-a6e8-156e1a351db0', '', '', '']
insoluble
['OVCAR3_dbgen1_K7845698_2_insoluble', 'b3f3d1d3-eb51-4615-a6e8-156e1a351db0', 'e45d66e9-abed-4d24-9dd8-8f074233c2d8', '01/01/2023', 'Lindsay', 'insoluble', 'extractionnote_here', '0', 'bca_here', '', 'storageform_here', 'storageamt_here', 'True', 'b3f3d1d3-eb51-4615-a6e8-156e1a351db0', '', '', '']
cytosol
['OVCAR3_dbgen1_K7843670_2_cytosol', 'f00d4d56-ac39-408b-af03-4a3a66c0e83d', 'e45d66e9-abed-4d24-9dd8-8f074233c2d8', '01/01/2023', 'Lindsay', 'cytosol', 'extractionnote_here', '0', 'bca_here', '', 'storageform_here', 'storageamt_here', 'True', 'f00d4d56-ac39-408b-af03-4a3a66c0e83d', '', '', '']
chromatin
['OVCAR3_dbgen1_K7843670_2_chromatin', 'f00d4d56-ac39-408b-af03-4a3a66c0e83d

['RH4_dbgen1_K7846403_2_chromatin', '9c6aff1b-88e3-465e-a3b9-3932b1de5763', 'e45d66e9-abed-4d24-9dd8-8f074233c2d8', '01/01/2023', 'Lindsay', 'chromatin', 'extractionnote_here', '0', 'bca_here', '', 'storageform_here', 'storageamt_here', 'True', '9c6aff1b-88e3-465e-a3b9-3932b1de5763', '', '', '']
insoluble
['RH4_dbgen1_K7846403_2_insoluble', '9c6aff1b-88e3-465e-a3b9-3932b1de5763', 'e45d66e9-abed-4d24-9dd8-8f074233c2d8', '01/01/2023', 'Lindsay', 'insoluble', 'extractionnote_here', '0', 'bca_here', '', 'storageform_here', 'storageamt_here', 'True', '9c6aff1b-88e3-465e-a3b9-3932b1de5763', '', '', '']
cytosol
['RH4_dbgen1_M2_2_cytosol', 'f2420dcf-d248-41b2-b967-5642e32ac2f8', 'e45d66e9-abed-4d24-9dd8-8f074233c2d8', '01/01/2023', 'Lindsay', 'cytosol', 'extractionnote_here', '0', 'bca_here', '', 'storageform_here', 'storageamt_here', 'True', 'f2420dcf-d248-41b2-b967-5642e32ac2f8', '', '', '']
chromatin
['RH4_dbgen1_M2_2_chromatin', 'f2420dcf-d248-41b2-b967-5642e32ac2f8', 'e45d66e9-abed-4d24-9

['IMR32_dbgen1_C2005693_2_chromatin', '5fa9d334-f5ca-4eac-8a29-5753fe1f80a1', 'e45d66e9-abed-4d24-9dd8-8f074233c2d8', '01/01/2023', 'Lindsay', 'chromatin', 'extractionnote_here', '0', 'bca_here', '', 'storageform_here', 'storageamt_here', 'True', '5fa9d334-f5ca-4eac-8a29-5753fe1f80a1', '', '', '']
insoluble
['IMR32_dbgen1_C2005693_2_insoluble', '5fa9d334-f5ca-4eac-8a29-5753fe1f80a1', 'e45d66e9-abed-4d24-9dd8-8f074233c2d8', '01/01/2023', 'Lindsay', 'insoluble', 'extractionnote_here', '0', 'bca_here', '', 'storageform_here', 'storageamt_here', 'True', '5fa9d334-f5ca-4eac-8a29-5753fe1f80a1', '', '', '']
cytosol
